In [39]:
import math

def get_tasks_for_thread(remainder: int, num_tasks_default: int, thread: int):
    num_tasks = num_tasks_default + 1 if thread < remainder else num_tasks_default
    threads_default = max(thread - remainder, 0)
    threads_default_plus = thread - threads_default
    start = threads_default * num_tasks_default + threads_default_plus * (num_tasks_default + 1)
    end = start + num_tasks - 1
    return num_tasks, start, end

def divide_tasks(tasks: int, threads: int):
    remainder = tasks % threads
    num_tasks_default = math.floor(tasks / threads)
    for thread in range(threads):
        num_tasks, start, end = get_tasks_for_thread(remainder, num_tasks_default, thread)
        print(f"Thread {thread}: tasks {start}-{end}, i.e. {num_tasks} tasks.")        

In [42]:
divide_tasks(5, 16)

Thread 0: tasks 0-0, i.e. 1 tasks.
Thread 1: tasks 1-1, i.e. 1 tasks.
Thread 2: tasks 2-2, i.e. 1 tasks.
Thread 3: tasks 3-3, i.e. 1 tasks.
Thread 4: tasks 4-4, i.e. 1 tasks.
Thread 5: tasks 5-4, i.e. 0 tasks.
Thread 6: tasks 5-4, i.e. 0 tasks.
Thread 7: tasks 5-4, i.e. 0 tasks.
Thread 8: tasks 5-4, i.e. 0 tasks.
Thread 9: tasks 5-4, i.e. 0 tasks.
Thread 10: tasks 5-4, i.e. 0 tasks.
Thread 11: tasks 5-4, i.e. 0 tasks.
Thread 12: tasks 5-4, i.e. 0 tasks.
Thread 13: tasks 5-4, i.e. 0 tasks.
Thread 14: tasks 5-4, i.e. 0 tasks.
Thread 15: tasks 5-4, i.e. 0 tasks.


In [7]:
20 % 3

2